In [11]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    model = "llama-3.3-70b-versatile" ,
    temperature = 0 ,
    groq_api_key = "<your_api_key>"
)

# Web Scraping

In [1]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
loader = WebBaseLoader("https://careers.nike.com/cdn-security-engineer/job/R-48004")

In [3]:
page_data = loader.load().pop().page_content
page_data

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCDN Security Engineer\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\nOpen Virtual Assistant\n\n\n\n\n\n\n\n\n\n\nHome\n\n\nCareer Areas\n\n\nTotal Rewards\n\n\nLife@Nike\n\n\nPurpose\n\n\n\n\n\n\n\n\n\n\nLanguage\n\n\n\n\n\nSelect a Language\n\n  Deutsch  \n  English  \n  Español (España)  \n  Español (América Latina)  \n  Français  \n  Italiano  \n  Nederlands  \n  Polski  \n  Tiếng Việt  \n  Türkçe  \n  简体中文  \n  繁體中文  \n  한국어  \n  日本語  \n\n\n\n\n\n\n\n\nCareers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nClose Menu\n\n\n\n\n\n\n\nCareers\n\n\n\n\n\n\nChat\n\n\n\n\n\n\n                                Home\n                            \n\n\n\n                                Career Areas\n                            \n\n\n\n                                Total Rewards\n                            \n\n\n\n                                Life@Nike\n                            \n\n\n\n        

# Prompt 

In [6]:
from langchain_core.prompts import PromptTemplate

In [31]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        For skills put it as an array of strings , each string representing one of the required skills.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

In [32]:
chain_extract = prompt_extract | llm

In [33]:
res = chain_extract.invoke(input = {'page_data' : page_data})
print(res.content)  

```json
{
  "role": "CDN Security Engineer",
  "experience": "At least five (5) or more years of relevant experience, Two (2) or more years of experience providing similar operations and sustainment support with Akamai or other CDN providers",
  "skills": [
    "CDN concepts",
    "Akamai's suite of tools and security offerings",
    "API security requirements",
    "Web attack vectors",
    "Threat indicators",
    "Cloud infrastructure",
    "Security",
    "AWS Certified Solutions Architect",
    "CISSP",
    "CEH"
  ],
  "description": "The CDN Security Engineer is an experienced technical professional who will play a critical role in designing, implementing, and sustaining Nike's cloud infrastructure and security posture, with a focus on Content Delivery Network (CDN) security."
}
```


In [34]:
type(res.content)
# this is in string , need to convert to json

str

In [35]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'CDN Security Engineer',
 'experience': 'At least five (5) or more years of relevant experience, Two (2) or more years of experience providing similar operations and sustainment support with Akamai or other CDN providers',
 'skills': ['CDN concepts',
  "Akamai's suite of tools and security offerings",
  'API security requirements',
  'Web attack vectors',
  'Threat indicators',
  'Cloud infrastructure',
  'Security',
  'AWS Certified Solutions Architect',
  'CISSP',
  'CEH'],
 'description': "The CDN Security Engineer is an experienced technical professional who will play a critical role in designing, implementing, and sustaining Nike's cloud infrastructure and security posture, with a focus on Content Delivery Network (CDN) security."}

In [36]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [23]:

import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [26]:
links = collection.query(query_texts = [ "experience in python"] , n_results = 2)
links
# chroma db can do a semantic search
# for python it brings two links which are related
# 'links': 'https://example.com/ml-python-portfolio'},
#    {'links': 'https://example.com/python-portfolio'}

{'ids': [['e2d44e3d-3e0e-4615-a814-a3fbf3af2b92',
   '9cde191f-4997-4dec-9c0e-22a914ad0ee7']],
 'embeddings': None,
 'documents': [['Machine Learning, Python, TensorFlow',
   'Python, Django, MySQL']],
 'uris': None,
 'data': None,
 'metadatas': [[{'links': 'https://example.com/ml-python-portfolio'},
   {'links': 'https://example.com/python-portfolio'}]],
 'distances': [[0.9968848007555503, 1.0578701721493369]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [37]:
job = json_res
job['skills']

['CDN concepts',
 "Akamai's suite of tools and security offerings",
 'API security requirements',
 'Web attack vectors',
 'Threat indicators',
 'Cloud infrastructure',
 'Security',
 'AWS Certified Solutions Architect',
 'CISSP',
 'CEH']

In [42]:
links = collection.query(query_texts = job['skills'] , n_results = 2).get('metadatas' , [])
links
# for each skill , we extract 2 relevant links

[[{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/kotlin-backend-port

# Email generation

In [47]:

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Ankit, a software engineer with 1 year of experience.
        You have epertise in Java , ReactJs , Springboot and Data structures & Algorithms.
        
        Your job is to write a cold email to the HR regarding the job mentioned above describing how you are suited for the job. 
        
        Also add the most relevant ones from the following links to showcase your portfolio: {link_list}
        Remember you are Ankit , a software engineer.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )


In [49]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description":str(job) , "link_list": links})
print(res.content)

Subject: Application for CDN Security Engineer Position

Dear Hiring Manager,

I am excited to apply for the CDN Security Engineer position at Nike, as advertised. Although I don't meet the exact experience requirements, I am confident that my strong foundation in software engineering and my enthusiasm for security and cloud infrastructure make me an ideal candidate for this role.

As a software engineer with 1 year of experience, I have developed expertise in Java, ReactJs, Springboot, and Data structures & Algorithms. I am eager to expand my skill set and take on new challenges, particularly in the areas of CDN security and cloud infrastructure. I am a quick learner, and I am excited about the opportunity to work with Akamai's suite of tools and security offerings.

I would like to showcase my portfolio, which demonstrates my proficiency in software development. Some relevant projects include:
- https://example.com/java-portfolio (Java portfolio)
- https://example.com/full-stack-js-p